Beware:
- duplicates
- images with markings
- number of useful pixels

Save as five csvs

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai
from fastai.vision import *
import os
from sklearn.model_selection import KFold
import warnings
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings("ignore")

fastai.__version__

'1.0.61'

In [2]:
benjamin_path = '/media/benjamin/Seagate Expansion Drive/content/prostate-cancer-grade-assessment/'
LABELS = '../../data/train.csv'
nfolds = 4

# Get a df of images without duplicates or markings

In [3]:
img_wo_duplicates = pd.read_csv('../data/images_wo_duplicates.csv')

In [4]:
img_duplicates = pd.read_csv('../data/five_hundred_dups.csv')
img_duplicates.columns = ['name']
img_duplicates.drop_duplicates(inplace=True)
img_duplicates['is_dup'] = 1

In [5]:
combined = pd.merge(img_wo_duplicates, img_duplicates, on='name', how='left')

In [6]:
df_without_marking_or_dups = combined[ (combined['is_dup'].isna()) & (combined['has_markings'].isna()) ]

# Remove images without masks 

In [7]:
list_dir = os.listdir(benjamin_path + 'train_label_masks')
names = [fn.split('_')[0] for fn in list_dir]
df_without_marking_or_dups = df_without_marking_or_dups[ df_without_marking_or_dups['name'].isin(names)]

# Get the isup grade for stratified Kfold

In [8]:
train_df = pd.read_csv('../data/train.csv').set_index('image_id')

In [9]:
validable = pd.merge(df_without_marking_or_dups['name'], train_df,
                     left_on='name', right_on='image_id', how='inner')
# make "negative" 0+0 to stratify on gleason score 
validable['gleason_score'] = validable['gleason_score'].apply(lambda x: '0+0' if x == 'negative' else x)
validable

,name,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0
...,...,...,...,...
9521,ffcee00fd033d3ece1408035a7fd4ea7,karolinska,1,3+3
9522,ffd2841373b39792ab0c84cccd066e31,radboud,0,0+0
9523,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5
9524,ffe06afd66a93258f8fabdef6044e181,radboud,0,0+0


# Make folds

In [10]:
splits = StratifiedKFold(n_splits=nfolds, random_state=42, shuffle=True)
splits = list(splits.split(validable,validable.gleason_score ))

In [11]:
# add splits to the dataframe 
folds_splits = np.zeros(len(validable)).astype(np.int)
for i in range(nfolds): folds_splits[splits[i][1]] = i
validable['split'] = folds_splits
validable

,name,data_provider,isup_grade,gleason_score,split
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,3
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,3
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,2
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,2
...,...,...,...,...,...
9521,ffcee00fd033d3ece1408035a7fd4ea7,karolinska,1,3+3,0
9522,ffd2841373b39792ab0c84cccd066e31,radboud,0,0+0,1
9523,ffdc59cd580a1468eac0e6a32dd1ff2d,radboud,5,4+5,1
9524,ffe06afd66a93258f8fabdef6044e181,radboud,0,0+0,1


# Re integrate other images

In [12]:
train_df.reset_index(inplace=True)
train_df.rename( {'image_id': 'name'}, axis=1, inplace=True)
df = pd.merge(validable,train_df, how='right')
df['split'] = df['split'].fillna(nfolds)

In [13]:
df['split'].value_counts()

1.0    2193
0.0    2186
3.0    2173
2.0    2161
4.0    1903
Name: split, dtype: int64

In [14]:
#problem 
df[ df['split'] == 4]['data_provider'].value_counts()

radboud       1851
karolinska      52
Name: data_provider, dtype: int64

In [15]:
df[ df['split'] != 3]['data_provider'].value_counts()

radboud       4300
karolinska    4143
Name: data_provider, dtype: int64

In [16]:
df[ df['split'] == 3]['data_provider'].value_counts()

karolinska    1313
radboud        860
Name: data_provider, dtype: int64

# Why

- all images with markings are from radboud
- almost all duplicates are from radboud too 

In [17]:
why = combined.merge(train_df)
why.head()

,name,names_wo_dups079,names_wo_dups085,has_markings,is_dup,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,NaN,NaN,NaN,NaN,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,NaN,NaN,NaN,NaN,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,NaN,NaN,NaN,NaN,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,NaN,NaN,NaN,NaN,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,001d865e65ef5d2579c190a0e0350d8f,001d865e65ef5d2579c190a0e0350d8f,NaN,NaN,karolinska,0,0+0


In [18]:
why[why['has_markings'].notnull()]['data_provider'].value_counts()

radboud       474
karolinska      1
Name: data_provider, dtype: int64

In [19]:
why[why['is_dup'].notnull()]['data_provider'].value_counts()

radboud       492
karolinska     50
Name: data_provider, dtype: int64

# Save folds as csv 

In [20]:
df.head(10)

,name,data_provider,isup_grade,gleason_score,split
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,1.0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,3.0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,3.0
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,2.0
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,2.0
5,002a4db09dad406c85505a00fb6f6144,karolinska,0,0+0,1.0
6,003046e27c8ead3e3db155780dc5498e,karolinska,1,3+3,3.0
7,0032bfa835ce0f43a92ae0bbab6871cb,karolinska,1,3+3,3.0
8,003a91841da04a5a31f808fb5c21538a,karolinska,1,3+3,3.0
9,003d4dd6bd61221ebc0bfb9350db333f,karolinska,1,3+3,0.0


In [21]:
# df.to_csv('../data/folds.csv', index=False)

# Note on splits
interestingly, all splits have about the same isup_grade distribution but not the fold 4 (with the markings and duplicates). Probably train without them for iterating to earn time

In [30]:
df[df['split'] == 1]['isup_grade'].value_counts()

1    606
0    493
2    299
4    276
3    261
5    258
Name: isup_grade, dtype: int64

In [31]:
df[df['split'] == 2]['isup_grade'].value_counts()

1    605
0    462
2    299
4    275
3    262
5    258
Name: isup_grade, dtype: int64

In [32]:
df[df['split'] == 3]['isup_grade'].value_counts()

1    606
0    474
2    298
4    275
3    262
5    258
Name: isup_grade, dtype: int64

In [33]:
df[df['split'] == 4]['isup_grade'].value_counts()

0    976
1    243
3    196
5    192
4    148
2    148
Name: isup_grade, dtype: int64